In [1]:
SANDBOX_NAME = # Sandbox Name
DATA_PATH = "/data/sandboxes/" + SANDBOX_NAME + "/data/"



# Spark ML

Cargamos un dataset con información de la distribución de los píxeles para cada una de las letras del alfabeto escritas en mayúsculas. El objetivo será predecir si el caracter en cuestión es una vocal o una consonante. En el proceso se aprenderán los pasos generales a seguir para solucionar un problema de este tipo con Spark ML.




### Crear SparkSession

In [2]:
# Respuesta

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()



### Cargar datos y comprobar schema

In [3]:
# Respuesta

letters = spark.read.csv(DATA_PATH+'data/letter.txt', sep=',', header=True, inferSchema=True)

In [4]:
# Respuesta

letters.printSchema()

root
 |-- x_box_integer: integer (nullable = true)
 |-- y_box_integer: integer (nullable = true)
 |-- width_integer: integer (nullable = true)
 |-- high_integer: integer (nullable = true)
 |-- onpix_integer: integer (nullable = true)
 |-- x_bar_integer: integer (nullable = true)
 |-- y_bar_integer: integer (nullable = true)
 |-- x2bar_integer: integer (nullable = true)
 |-- y2bar_integer: integer (nullable = true)
 |-- xybar_integer: integer (nullable = true)
 |-- x2ybr_integer: integer (nullable = true)
 |-- xy2br_integer: integer (nullable = true)
 |-- x_ege_integer: integer (nullable = true)
 |-- xegvy_integer: integer (nullable = true)
 |-- y_ege_integer: integer (nullable = true)
 |-- yegvx_integer: integer (nullable = true)
 |-- class: string (nullable = true)





### Crear nueva variable objetivo

La variable objetivo ahora mismo es cada una de las letras del abecedario en mayúscula. Se crea nueva variable con el nombre 'flag' que tome el valor 1 si se trata de una vocal y 0 en caso contrario, para convertir el problema en un problema de clasificación binaria.

In [5]:
# Respuesta

import pyspark.sql.functions as F
from pyspark.sql.types import DoubleType

letters = letters.withColumn('tag', F.udf(lambda value: 1. if value in ['A', 'E', 'I', 'O', 'U'] else 0., DoubleType())(F.col('class')))



### Primeros pasos

Primeros pasos: nulos y vector assembler



Empezaremos con la comprobación y eliminación nulos. Deberíamos comprobar qué es lo que estamos eliminando, pero para la realización de este ejemplo, simplemente los eliminamos todos.

In [6]:
# Respuesta

for column in letters.columns:
    letters = letters.where(F.col(column).isNotNull())



Tras remover todos los nulos, usamos el VectorAssembler con todas las variables excepto las objetivo (la nueva variable objetivo *tag* y la original *class*).

Asumimos aquí que todas las variables son numéricas y que las queremos usar como input para nuestro modelo.

In [7]:
# Respuesta

from pyspark.ml.feature import VectorAssembler

vectorassembler = VectorAssembler(inputCols=[element for element in letters.columns if element != 'tag' and element !='class'], outputCol='assembled_features')
letters = vectorassembler.transform(letters)



### Selección de variables



Vamos a hacer lo que vimos en el Notebook de selección de variables

Vamos a tomar un número de semillas aleatorias, y vamos a calcular la importancia de variables para cada una de las semillas. Una vez hecho esto, vamos a tomar como variables importantes aquellas que aparezcan en cada una de las iteraciones con las distintas semillas.

Empezamos generando la lista de semillas aleatorias.

In [8]:
# Respuesta

from pyspark.ml.classification import RandomForestClassifier

random_seed = 4 
num_iter = 10 #number of seeds we will use

import random

# Random seed for replicability, so every time we run it, it returns the same random values
random.seed(random_seed)

random_seeds=[] #list containing our random seeds

# Get num_iter (10 in this example) random seeds and append them in our list random_seeds
while len(set(random_seeds)) < num_iter:
    random_seeds.append(random.randint(0,10000))



Creamos una lista *features_random_seed* que es una lista de listas, dónde cada lista (1 por semilla, es decir, una por random forest) tiene tuplas con los nombres de variables y las importancias de las variables que explican el 95% de la importancia.

In [9]:
# Respuesta

features_random_seed = [] 

for random_seed in random_seeds:
    rf = RandomForestClassifier(featuresCol=vectorassembler.getOutputCol(), labelCol='tag', seed = random_seed )
    rf_model = rf.fit(letters)
    
    # We get the importances for the seed in this iteration
    # We save the index to be able to get the variable name later
    importances = [(index, value) for index, value in enumerate(rf_model.featureImportances.toArray().tolist())]

    # Sort from higher to lower relevance
    importances = sorted(importances, key=lambda value: value[1], reverse=True)
    
    # We keep the ones that explain the 95% of importance regarding the target variable

    compt = 0
    important_features =[]
    for element in importances:
        if compt < 0.95:
            compt += element[1]
            important_features.append((vectorassembler.getInputCols()[element[0]], element[1]))
    features_random_seed.append(important_features)



Creamos una lista *features_all_seeds* que es una lista de variables que aparecieron como importantes en cada iteración de las semillas

In [10]:
# Respuesta

flat_features = [feature for one_seed in features_random_seed for feature in one_seed]
features = [element[0] for element in flat_features]
# We transform our list of lists to just one list 

# We use Counter to get the variables which appear as important in every iteration
# Elements are stored as dictionary keys and their counts are stored as dictionary values.
from collections import Counter

features_all_seeds = [element[0] for element in Counter(features).items() if element[1] == num_iter]



Ahora creamos *dicitonary_importances* que tendrá el nombre de cada variable (que apareció como importante en todos los RF) y la media de la importancia que obtuvo para todas las semillas

In [11]:
# Respuesta

import numpy as np

dictionary_importances = {}

for feature in features_all_seeds: # has the important variables which appeared in all RF
    dictionary_importances[feature] = []
    
    # Features_random_seed is a list of lists (1/RF), where each list has tuples with variables and importances
    for values in features_random_seed:
        for element in values: # values is each list of a RF with tuples
            if element[0] == feature: # element is each tuple (variable, importance)
                dictionary_importances[feature].append(element[1]) # append importance values of each RF
                break
    dictionary_importances[feature] = np.mean(dictionary_importances[feature]) # get the mean of the importance

dictionary_importances = sorted(dictionary_importances.items(), key=lambda value: value[1], reverse=True)
dictionary_importances

[('xy2br_integer', 0.17033443008368546),
 ('y_bar_integer', 0.14201330587864597),
 ('width_integer', 0.10679107744754845),
 ('x_ege_integer', 0.10426602229681688),
 ('x2ybr_integer', 0.08891062630343921),
 ('x2bar_integer', 0.08847049838692729),
 ('y2bar_integer', 0.08702226475991735),
 ('xegvy_integer', 0.06031182687195572),
 ('y_ege_integer', 0.04687976572650596)]



Mostrar las variables más importantes

In [12]:
# Respuesta

features_all_seeds

['xegvy_integer',
 'width_integer',
 'y2bar_integer',
 'x2bar_integer',
 'y_bar_integer',
 'xy2br_integer',
 'y_ege_integer',
 'x2ybr_integer',
 'x_ege_integer']



Hemos visto las variables más importantes, podríamos intentar usar sólo esas para nuestro modelo viendo cuál es el resultado de realizar eso. En esta caso, vamos a continuar con todas.

Estandarizamos los valores y lanzamos un modelo de clasificación dentro de un Pipeline



**Regresión Logística** 

Vamos a hacer primero el vector assembler para alimentar este al StandardScaler, y la salida del mismo será el input para nuestro modelo.

Esto lo hacemos utilizando un Pipeline como ya vimos anteriormente (haciendo el fit y transform sólo en el pipeline).

In [13]:
# Respuesta

from pyspark.ml import Pipeline
from pyspark.ml.feature import StandardScaler
from pyspark.ml.classification import LogisticRegression

vector_assembler = VectorAssembler(inputCols=features_all_seeds, outputCol='assembled_important_features')
standard_scaler = StandardScaler(inputCol=vector_assembler.getOutputCol(), outputCol='standardized_features')
log_reg = LogisticRegression(featuresCol=standard_scaler.getOutputCol(), labelCol='tag')

pipeline_log_reg = Pipeline(stages=[vector_assembler, standard_scaler, log_reg])

letters_train, letters_test = letters.randomSplit([0.8,0.2], seed=4)

pipeline_model_log_reg = pipeline_log_reg.fit(letters_train)

letters_test_log_reg = pipeline_model_log_reg.transform(letters_test)

letters_test_log_reg.show()


+-------------+-------------+-------------+------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-----+---+--------------------+----------------------------+---------------------+--------------------+--------------------+----------+
|x_box_integer|y_box_integer|width_integer|high_integer|onpix_integer|x_bar_integer|y_bar_integer|x2bar_integer|y2bar_integer|xybar_integer|x2ybr_integer|xy2br_integer|x_ege_integer|xegvy_integer|y_ege_integer|yegvx_integer|class|tag|  assembled_features|assembled_important_features|standardized_features|       rawPrediction|         probability|prediction|
+-------------+-------------+-------------+------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-----+---+--------------------+-------------------------



**RandomForest**

Hacemos lo mismo que con la regresión logística pero utilizando como modelo el Random Forest.

In [14]:
# Respuesta

from pyspark.ml.classification import RandomForestClassifier

vector_assembler = VectorAssembler(inputCols=features_all_seeds, outputCol='assembled_important_features')
standard_scaler = StandardScaler(inputCol=vector_assembler.getOutputCol(), outputCol='standardized_features')
rf = RandomForestClassifier(featuresCol=standard_scaler.getOutputCol(), labelCol='tag')

pipeline = Pipeline(stages=[vector_assembler, standard_scaler, rf])

pipeline_model_rf = pipeline.fit(letters_train)

letters_test_rf = pipeline_model_rf.transform(letters_test)

letters_test_rf.show()

+-------------+-------------+-------------+------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-----+---+--------------------+----------------------------+---------------------+--------------------+--------------------+----------+
|x_box_integer|y_box_integer|width_integer|high_integer|onpix_integer|x_bar_integer|y_bar_integer|x2bar_integer|y2bar_integer|xybar_integer|x2ybr_integer|xy2br_integer|x_ege_integer|xegvy_integer|y_ege_integer|yegvx_integer|class|tag|  assembled_features|assembled_important_features|standardized_features|       rawPrediction|         probability|prediction|
+-------------+-------------+-------------+------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-----+---+--------------------+-------------------------



### Evaluar modelos para decidir cuál predice mejor



### Regresión Logística



Importar librerías necesarias. Podemos observar que usamos también mllib (recordemos que con dicha librería debemos trabajar con rdd en lugar de dataframe) para obtener el objeto MulticlassMetrics que usaremos para calcular el recall, precision, f1 score y la matriz de confusión.

In [15]:
# Respuesta

from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics



AUC

In [16]:
# Respuesta

auc = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction', labelCol='tag', metricName='areaUnderROC')
auc.evaluate(letters_test_log_reg)

0.7269502483778738



Otras métricas

In [17]:
# Respuesta

metrics = MulticlassMetrics(letters_test_log_reg.select('prediction', 'tag').rdd)

recall = metrics.recall(label=1)
precision = metrics.precision(label=1)
f1 = metrics.fMeasure()
confusion_matrix = metrics.confusionMatrix()

print("Recall: {}".format(recall))
print("Precision: {}".format(precision))
print("f1: {}".format(f1))
print("Confusion matrix: {}".format(confusion_matrix))

/opt/spark/dist/python/pyspark/mllib/evaluation.py:262: UserWarning: Deprecated in 2.0.0. Use accuracy.
  warnings.warn("Deprecated in 2.0.0. Use accuracy.")


Recall: 0.061124694376528114
Precision: 0.3816793893129771
f1: 0.7878030492376906
Confusion matrix: DenseMatrix([[3102.,   81.],
             [ 768.,   50.]])




### Random Forest



Realizamos lo mismo que con la regresión logística, calculando: AUC, otras métricas (recall, precision, f1 score) y matriz de confusión

In [18]:
# Respuesta

auc = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction', labelCol='tag', metricName='areaUnderROC')
auc.evaluate(letters_test_rf)

0.8787438155174935

In [19]:
# Respuesta

metrics = MulticlassMetrics(letters_test_rf.select('prediction', 'tag').rdd)

recall = metrics.recall(label=1)
precision = metrics.precision(label=1)
f1 = metrics.fMeasure()
confusion_matrix = metrics.confusionMatrix()

print("Recall: {}".format(recall))
print("Precision: {}".format(precision))
print("f1: {}".format(f1))
print("Confusion matrix: {}".format(confusion_matrix))

/opt/spark/dist/python/pyspark/mllib/evaluation.py:262: UserWarning: Deprecated in 2.0.0. Use accuracy.
  warnings.warn("Deprecated in 2.0.0. Use accuracy.")


Recall: 0.26894865525672373
Precision: 0.9777777777777777
f1: 0.8492876780804799
Confusion matrix: DenseMatrix([[3178.,    5.],
             [ 598.,  220.]])
